# Imports

In [0]:
import numpy
import pandas
import keras
import tensorflow.contrib.eager as tfe
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.utils import np_utils
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.utils import shuffle
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras import Input

Using TensorFlow backend.


# fixing random seed

In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# loading dataset

In [0]:
# load dataset
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

# Encoding labels

In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


In [0]:
X.shape

(150, 4)

# Model Subclassing

## Tuned Model

In [0]:
class MyTunedModel(keras.Model):  # change
  
  def __init__(self):
    super(MyTunedModel,self).__init__('my_tuned')   # change     
    self.dense1 = keras.layers.Dense(8, activation='relu')
    self.dense2 = keras.layers.Dense(10, activation='relu')   
    self.dense3 = keras.layers.Dense(10, activation='relu')   
    self.dense4 = keras.layers.Dense(10, activation='relu')   # change
    self.dense5 = keras.layers.Dense(3, activation='softmax')
    
  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)      # change
    x = self.dense3(x) 
    x = self.dense4(x)  
    return self.dense5(x)
    
def tuned_subclass_model():  # change
  inputs = keras.Input(shape=(4,))
  mymodel = MyTunedModel()        # change
  outputs = mymodel.call(inputs)
    
  model = keras.Model(inputs,outputs)
    
# compiling model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

  return model

estimator = KerasClassifier(build_fn=tuned_subclass_model, epochs=100, batch_size=5, verbose=0)    # change

kfold = KFold(n_splits=10, shuffle=True, random_state=None)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("tuned_subclass_model: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))  # change

tuned_subclass_model: 98.67% (2.67%)
